In [ ]:
import sys
import numpy as np
from numpy.linalg import inv
from numpy.random import RandomState, SeedSequence, MT19937
from scipy.interpolate import interp1d
from scikits.odes import dae
import matplotlib.pyplot as plt
import seaborn as sns

if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import OU
from filter_OU_inputs import run_welch

#### Base units
Base apparent power and frequency as defined in PowerFactory. Base torque is computed accordingly.

In [ ]:
S_base = 1e6                # [VA] base apparent power
V_base = 10e3               # [V] base voltage
ω_base = 2 * np.pi * 50     # [Hz] base frequency
T_base = S_base / ω_base    # [Nm] base torque
Z_base = V_base**2 / S_base # [Ω] base impedance
Y_base = 1 / Z_base         # [S] base admittance
print('S_base = {:g} MVA'.format(S_base * 1e-6))
print('V_base = {:g} kV'.format(V_base * 1e-3))
print('F_base = {:g} Hz'.format(ω_base / (2 * np.pi)))
print('T_base = {:g} Nm'.format(T_base))
print('Z_base = {:g} Ω'.format(Z_base))
print('Y_base = {:g} S'.format(Y_base))

Load the power flow data:

In [ ]:
AC_fname = '../data/SM_with_load/adynamic_load_const_S/LD1/SM_with_load_AC.npz'
data = np.load(AC_fname, allow_pickle=True)
PF = data['PF_without_slack'].item()
PF_load = PF['loads']['LD1']
PF_gen = PF['SMs']['G1']
PF_bus = PF['buses']['Bus1']

#### Generator parameters

In [ ]:
# with the following value of generator apparent power, the Z_base of the
# generator is equivalent to the system one (i.e., the one computed above
# using V_base and S_base
S_gen = 1.732051e6            # [MVA] apparent power of the generator
tag = 8                       # [s] acceleration time constant
H = tag / 2                   # [s] inertia constant
E_kin = H * S_gen             # [J] kinetic energy of the generator
J = 2 * E_kin / (ω_base ** 2) # [kgm2] moment of inertia of the generator

I_base_gen = S_gen / (np.sqrt(3) * V_base)
assert abs(PF_gen['I'] * 1e3 / PF_gen['i'] - I_base_gen) < 1e-4
Z_base_gen = V_base / I_base_gen              # [Ω]
iGr = PF_gen['ir'] * I_base_gen               # [A]
iGi = PF_gen['ii'] * I_base_gen               # [A]
assert abs(PF_gen['I'] * 1e3 - np.abs(iGr + 1j * iGi)) < 1e-4

rstr, xstr = 0.2, 0.4                         # [pu] stator parameters
R_gen = rstr * Z_base_gen                     # [Ω]
X_gen = xstr * Z_base_gen                     # [Ω]
Z_gen = R_gen + 1j * X_gen                    # [Ω]

print('============ SM ===========')
print('       tag = {:g} s'.format(tag))
print('         H = {:g} s'.format(H))
print('     E_kin = {:g} MJ'.format(E_kin*1e-6))
print('         J = {:g} kgm2'.format(J))
print('I_base_gen = {:.2f} A'.format(I_base_gen))
print('Z_base_gen = {:.2f} Ω'.format(Z_base_gen))
print('     I_gen = {:.2f} A'.format(iGr + 1j * iGi))
print('     Z_gen = {:.2f} Ω'.format(Z_gen))

#### Load parameters

In [ ]:
assert abs(V_base - PF_load['Vl'] * 1e3 / PF_load['u']) < 1e-10
P_load = PF_load['P'] * 1e6   # [W]
Q_load = PF_load['Q'] * 1e6   # [VAR]
S_load = P_load + 1j * Q_load # [VA]
V_load = (PF_load['ur'] + 1j * PF_load['ui']) * V_base
I_load = (S_load / (V_load * np.sqrt(3))).conjugate()
Z_load = V_load / I_load
iLr, iLi = float(I_load.real), float(I_load.imag)
assert abs(PF_load['ir'] * np.abs(I_load) - iLr) < 1e-6, 'Real part of the current does not match'
assert abs(PF_load['ii'] * np.abs(I_load) - iLi) < 1e-6, 'Imaginary part of the current does not match'
print('========== LOAD ========')
print('S_load = {:g} MVA'.format(S_load * 1e-6))
print('I_load = {:.2f} A'.format(I_load))
print('Z_load = {:.2f} Ω'.format(Z_load))

#### Bus parameters

In [ ]:
assert abs(V_base - PF_bus['Vl'] * 1e3 / PF_bus['u']) < 1e-10
uBr = PF_bus['ur'] * V_base
uBi = PF_bus['ui'] * V_base
assert abs(PF_bus['Vl'] * 1e3 - np.abs(uBr + 1j * uBi)) < 1e-10
print('========= BUS ========')
print('V_bus = {:.2f} kV'.format((uBr + 1j * uBi) * 1e-3))

#### First algebraic equation:

In [ ]:
Ggnd = 1e-4                  # couldn't figure out the units of measure
Ggnd_S = float(iGr - iLr) / uBr   # [S]
iG = iGr + 1j * iGi
iL = iLr + 1j * iLi
uB = uBr + 1j * uBi
assert abs(iGr - (iLr + Ggnd_S * uBr)) < 1e-6
assert abs(iGi - (iLi + Ggnd_S * uBi)) < 1e-6

#### Second algebraic equation:

In [ ]:
den = np.sqrt(3) * (uBr ** 2 + uBi ** 2)
assert np.abs(iLr - (P_load * uBr + Q_load * uBi) / den) < 1e-10
assert np.abs(iLi - (-Q_load * uBr + P_load * uBi) / den) < 1e-10

#### Third algebraic equation:

In [ ]:
# the voltage of the ideal generator in the synchronous machine
ve = uB + Z_gen * iG
E0, ϕG = float(np.abs(ve)), float(np.angle(ve))
assert np.abs(uBr + R_gen * iGr - X_gen * iGi - E0 * np.cos(ϕG)) < 1e-10
assert np.abs(uBi + X_gen * iGr + R_gen * iGi - E0 * np.sin(ϕG)) < 1e-10

### Torques
General parameters:

In [ ]:
n = 1     # [pu]
nref = 1  # [pu]
cosn = 1  # rated power factor

#### Electrical torque

In [ ]:
ut = PF_gen['ur'] + 1j * PF_gen['ui']                    # [pu]
it = PF_gen['ir'] + 1j * PF_gen['ii']                    # [pu]
ψstr = (ut + rstr * it) / (1j * n)                       # [pu]
te = (it.imag * ψstr.real - it.real * ψstr.imag) / cosn  # [pu]
Te = te * T_base                                         # [Nm]
print('Stator flux: {:g} p.u.'.format(ψstr))
print('Electrical torque: {:g} p.u.'.format(te))
print('Electrical torque: {:g} Nm'.format(Te))

#### Mechanical torque

In [ ]:
dpu   = 0
xmdm  = 0
addmt = 0
pt = (te + dpu * n + xmdm) * n
tm = pt / n - (xmdm + dpu * n - addmt)
Tm = tm * T_base
print('Mechanical torque: {:g} p.u.'.format(tm))
print('Mechanical torque: {:g} Nm'.format(Tm))

#### Damping torque

In [ ]:
dkd, dpe = 0, 0
tdkd = dkd * (n - nref)
tdpe = dpe / n * (n - nref)
Tdkd, Tdpe = tdkd * T_base, tdpe * T_base
print('Damping torques: ({:g},{:g}) p.u.'.format(tdkd, tdpe))
print('Damping torques: ({:g},{:g}) Nm.'.format(Tdkd, Tdpe))

#### First ODE:

In [ ]:
assert (tm - te - tdkd - tdpe) / tag * ω_base == 0
assert (Tm - Te - Tdkd - Tdpe) / tag * ω_base == 0

### The system of DAEs

In [ ]:
class Parameters (object):
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)
    def __str__(self):
        par_names = 'w_base', 'V_base', 'T_base', 'P_load', 'Q_load', 'Ggnd', 'rstr', \
            'R_gen', 'X_gen', 'tag', 'E0', 'phiG', 'cosn', 'I_base_gen', 'pt', 'xmdm', \
            'dpu', 'addmt'
        return '\n'.join(map(str, [getattr(self, name) for name in par_names]))

def fun(t, y, ydot, params, res):
    w, phi, uBr, uBi, iLr, iLi, iGr, iGi = y
    wdot, phidot = ydot[:2]
    den = np.sqrt(3) * (uBr ** 2 + uBi ** 2)

    # mechanical torque
    n = w / params.w_base
    tm = params.pt / n - (params.xmdm + params.dpu * n - params.addmt)
    Tm = tm * params.T_base

    # electrical torque
    n = 1 # neglect rotor speed variations
    ut = (uBr + 1j * uBi) / params.V_base                          # [p.u.]
    it = (iGr + 1j * iGi) / params.I_base_gen                      # [p.u.]
    ψstr = (ut + params.rstr * it) / (1j * n)                      # [p.u.]
    te = (it.imag * ψstr.real - it.real * ψstr.imag) / params.cosn # [p.u.]
    Te = te * params.T_base                                        # [Nm]

    res[0] = wdot - (tm - te) / params.tag * params.w_base
    res[1] = phidot
    res[2] = iGr - (iLr + params.Ggnd * uBr)
    res[3] = iGi - (iLi + params.Ggnd * uBi)
    res[4] = iLr - ( params.P_load * uBr + params.Q_load * uBi) / den
    res[5] = iLi - (-params.Q_load * uBr + params.P_load * uBi) / den
    res[6] = uBr + params.R_gen * iGr - params.X_gen * iGi - params.E0 * np.cos(params.phiG)
    res[7] = uBi + params.X_gen * iGr + params.R_gen * iGi - params.E0 * np.sin(params.phiG)

#### Apply a step in the active power of the load

The figures below taken from PowerFactory refer to a step in the load active power of +1%.

First of all, check that at the PF the system is at an equilibrium:

In [ ]:
params = Parameters(w_base=ω_base, V_base=V_base, T_base=T_base, P_load=P_load,
                    Q_load=Q_load, Ggnd=Ggnd_S, rstr=rstr, R_gen=R_gen, X_gen=X_gen,
                    tag=tag, E0=E0, phiG=ϕG, cosn=cosn, I_base_gen=I_base_gen,
                    pt=pt, xmdm=xmdm, dpu=dpu, addmt=addmt)
y0 = np.array([ω_base, 0, uBr, uBi, iLr, iLi, iGr, iGi], dtype=float)
ydot0 = np.zeros(8, dtype=float)
res = np.zeros(8, dtype=float)
fun(0, y0, ydot0, params, res)
res

Integrate the system:

In [ ]:
algebraic_vars_idx = [2, 3, 4, 5, 6, 7]
solver = dae('ida', lambda t,y,ydot,res: fun(t, y, ydot, params, res), 
             compute_initcond='yp0',
             first_step_size=1e-12,
             atol=1e-4,
             rtol=1e-4,
             algebraic_vars_idx=algebraic_vars_idx,
             compute_initcond_t0=10,
             old_api=False)
dt = 5e-3
t0, t1 = 0, 60
t_eval = np.r_[t0 : t1 : dt]
sol1 = solver.solve(t_eval, y0, ydot0)

Change the value of the load:

In [ ]:
params.P_load *= 1.01
y0 = sol1.values.y[-1,:]
ydot0 = sol1.values.ydot[-1,:]
fun(t1, y0, ydot0, params, res)
res

Perform a second integration:

In [ ]:
t0, t1 = t1, t1+300
t_eval = np.r_[t0 : t1 : dt]
sol2 = solver.solve(t_eval, y0, ydot0)

### Results comparison

#### Generator speed

PowerFactory results:

<img src="../data/SM_with_load/adynamic_load_const_S/LD1/generator_speed.png" width=700px>

In [ ]:
ω_1 = sol1.values.y[:,0] / ω_base
ω_2 = sol2.values.y[:,0] / ω_base
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(sol1.values.t, ω_1, 'k', lw=1)
ax.plot(sol2.values.t, ω_2, 'r', lw=1)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Generator speed [p.u.]')
sns.despine()
fig.tight_layout()

#### Line-ground voltage

PowerFactory results:

<img src="../data/SM_with_load/adynamic_load_const_S/LD1/bus_voltage_load_step.png" width=700px>

In [ ]:
ur, ui = sol1.values.y[:,2], sol1.values.y[:,3]
V_bus_1 = np.abs(ur + 1j * ui) / np.sqrt(3) * 1e-3
ur, ui = sol2.values.y[:,2], sol2.values.y[:,3]
V_bus_2 = np.abs(ur + 1j * ui) / np.sqrt(3) * 1e-3
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(sol1.values.t, V_bus_1, 'k', lw=1)
ax.plot(sol2.values.t, V_bus_2, 'r', lw=1)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Line-Ground voltage [kV]')
sns.despine()
fig.tight_layout()
print('Line-ground voltage before load step: {:g} kV'.format(V_bus_1[-1]))
print('Line-ground voltage  after load step: {:g} kV'.format(V_bus_2[-1]))

#### Load and generator currents

PowerFactory results:

<img src="../data/SM_with_load/adynamic_load_const_S/LD1/load_generator_currents_load_step.png" width=700px>

In [ ]:
iLr, iLi = sol1.values.y[:,4], sol1.values.y[:,5]
I_load_1 = np.abs(iLr + 1j * iLi)
iLr, iLi = sol2.values.y[:,4], sol2.values.y[:,5]
I_load_2 = np.abs(iLr + 1j * iLi)
iGr, iGi = sol1.values.y[:,6], sol1.values.y[:,7]
I_gen_1 = np.abs(iGr + 1j * iGi)
iGr, iGi = sol2.values.y[:,6], sol2.values.y[:,7]
I_gen_2 = np.abs(iGr + 1j * iGi)
fig,ax = plt.subplots(2, 1, figsize=(5,5), sharex=True)
ax[0].plot(sol1.values.t, I_load_1, 'k', lw=1)
ax[0].plot(sol2.values.t, I_load_2, 'r', lw=1)
ax[1].plot(sol1.values.t, I_gen_1, 'k', lw=1)
ax[1].plot(sol2.values.t, I_gen_2, 'r', lw=1)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('Load current [A]')
ax[1].set_ylabel('Generator current [A]')
sns.despine()
fig.tight_layout()
print('Load current before load step: {:g} kV'.format(I_load_1[-1]))
print('Load current  after load step: {:g} kV'.format(I_load_2[-1]))
print('Generator current before load step: {:g} kV'.format(I_gen_1[-1]))
print('Generator current  after load step: {:g} kV'.format(I_gen_2[-1]))